<a href="https://colab.research.google.com/github/bhavini-jeloka/me604-robotics/blob/main/ME604_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Importing Files

In [ ]:
import numpy as np
from math import sqrt, atan2
import matplotlib.pyplot as plt

##Path Planning Algorithm

In [ ]:
# known: robot radius R
# known: obstacle radius r
# check the boolean functions
# check the 'self' and all

class PathPlanning:

  def __init__(self):
    self.R = 0.01
    self.r = 0.1
    self.epsilon = 0.0001
    self.obstaclesList = []
    self.start = None
    self.pseudoGoal = None
    self.stepSize = 0.1
    self.N = 10
    self.workspace = None


  def generateWP(self): 
    self.V = [self.start, self.pseudoGoal]
    # generate waypoints
    while len(self.V)<self.N:
      q = self.generateSample(self.workspace) # workspace obstacle 
      if not self.insideObstacle(q) and q not in self.V: # configuration space
        self.V.append(q)
    # return self.V


  def generateGraph(self): # dictionary from q what all points are accessible
    self.generateWP()
    self.graph = {}
    for q in self.V:
      E = []
      for p in self.V:
        if p == q:
          continue
        connect = self.generatePath(q,p)  # from q to p
        if self.checkObstacleCollision(connect):
          E.append(p)
      self.graph[q] = E
    # return self.graph


  def computeDistanceTwoPoints(self, p1, p2):  
      x1, y1 = p1[0], p1[1]
      x2, y2 = p2[0], p2[1]
      return sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)
  

  def angleBetweenVectors(self, v, w):
    v = np.array(v)
    w = np.array(w)
    angle = 0
    if np.linalg.norm(v)*np.linalg.norm(w) != 0.0:
        unit_vector_1 = v / np.linalg.norm(v)
        unit_vector_2 = w / np.linalg.norm(w)
        dot_product = np.dot(unit_vector_1, unit_vector_2)
        if abs(dot_product) > 1:
            dot_product = np.sign(dot_product)*1
        angle = np.arccos(dot_product)
    return angle


  def insideObstacle(self, q):
    for obstacle in self.obstaclesList:
      if self.computeDistanceTwoPoints(obstacle, q)<(self.r+self.R+self.epsilon):
        return True
    return False


  def generatePath(self, q, p):
    pq = np.array(p)-np.array(q)
    pq_hat = (pq)/np.linalg.norm(pq)
    path = []
    stepTaker = np.array(q)
    while self.computeDistanceTwoPoints(stepTaker, p) < self.stepSize:
      path.append(stepTaker)
      stepTaker = stepTaker + self.stepSize*pq_hat


  def checkObstacleCollision(self, path):
    for point in path:
      if self.insideObstacle(point):
        return True
    return False


  def generateSample(self):
    # as of now rectangular workspace parallel to x-y plane [[x_min, x_max], [y_min, y_max]]
    xLimit, yLimit = self.workspace
    xGen = np.random.uniform(xLimit[0], xLimit[1])
    yGen = np.random.uniform(yLimit[0], yLimit[1])
    return (xGen, yGen)


  def heuristic(self, q):
    self.generateGraph()
    deviationFromGoal = []
    direction2goal = [self.pseudoGoal[0]-self.q[0],
                      self.pseudoGoal[1]-self.q[1]]
    for possibleNextPoint in self.graph[q]:
        possibleNextStep = [self.possibleNextPoint[0]-self.q[0],
                            self.possibleNextPoint[1]-self.q[1]]
        deviationFromGoal.append(self.angleBetweenVectors(direction2goal, possibleNextStep))
    i = np.argmin(deviationFromGoal)
    return self.graph[q][i]


  def pathSearch(self):
    currentPos = self.start
    optPath = [self.start]
    while self.pseudoGoal not in optPath:
      newPos = self.heuristic(currentPos)
      optPath.append(newPos)
      currentPos = newPos
    return optPath


## Git Authentication

In [16]:
!ssh-keygen -t rsa -b 4096

Generating public/private rsa key pair.
Enter file in which to save the key (/root/.ssh/id_rsa): 
Enter passphrase (empty for no passphrase): 
Enter same passphrase again: 
Your identification has been saved in /root/.ssh/id_rsa.
Your public key has been saved in /root/.ssh/id_rsa.pub.
The key fingerprint is:
SHA256:66UZXNv9vqcYg3gOQV9JxNZSjmOuxvfGw27JOGpXZ2g root@b6f97ca8773e
The key's randomart image is:
+---[RSA 4096]----+
|           ooo.  |
|           .++.  |
|        .  .*..  |
|       . . + .   |
|        S o .  . |
|       . * = .E o|
|        * O =B.+ |
|       . X..+=O..|
|        +.oo.=+==|
+----[SHA256]-----+


In [17]:
!ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts

# github.com:22 SSH-2.0-babeld-4f04c79d


In [18]:
# Add your SSH keys to github from the output of the cell below
!cat /root/.ssh/id_rsa.pub

ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQCxm3Q/Ga4jdwii9b7ICXpuRbX0VynhoN9p/fEilF1+XQmXSmwRiyh12CE0wYHvJQEvvEJ9W+YTYv6xqVQjyslDW1EWB/c6pIvjZuOzp4EenObnYT3Wb+3ACQ8ukA0hdwb14E3QWpIzBD3O9RgzGbC4h/F+Q2DFnkdZXy4Vvk47v7AzZfzkw/9aZ+cRMKgQXPGN/Ei+mXF4ztevm3rh6HN8hJaWIZYwahNAYc2m9ApeiiICyTVVnyuipSRTe5XJvLlMAKzWoPHs+b8q9zw2JqS4shryoRPSaIR379pTsw2e6OjMIeP3xyTxo2wLRnvQzACOpar6/IUilFzyKbaJWpoe2kUofzvp2yOMPbpxQfS40BgUsVLlm4/UhJsjuiVZShsPay50OZaTx2Cw+HuE3I8Vut/3ByebX/9dvrVZPrpwwbpelIsqXnMrAt6+OZ/nRwsa1GkRRqZsOhzyEM7ZFL1V10HvG+DSQl4pnT637Z3ShWY4ftRP0YHRp9DEH+g7XOR4glJWbgUinOFnRO7zjF+PuQU3HpS1mWV+vYNOWnRVLArISd1aXjYFnNLDQBK0ndYh6Bro5261PqrlxwI/5HvDHAIu0korOBAxZ0dqwKtUCrWUwHZiZJJlqiHT+O8IOujvJjFkFRbuJz9LiMPLYvUK1m7w1UKCNhrg/NUxnTUUhw== root@b6f97ca8773e


In [19]:
# just a test
!ssh -T git@github.com

Hi bhavini-jeloka! You've successfully authenticated, but GitHub does not provide shell access.


In [22]:
# Configuring the user data
!git config --global user.email "jelokab@gmail.com"
!git config --global user.name "bhavini-jeloka"

In [24]:
! git clone git@github.com:bhavini-jeloka/me604-robotics.git

Cloning into 'me604-robotics'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (3/3), done.


In [28]:
%cd me604-robotics/

/content/me604-robotics


In [35]:
! git add .

In [36]:
! git commit -m "Add path planning algorithm"

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
